<a href="https://colab.research.google.com/github/chuntsehsu/Edge-detect-AOI/blob/main/ORVP_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 參考資料

In [ ]:
# jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

# 程式代碼

## 特徵工程

In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings

In [2]:
# data_dir = '/content/ORVP/'
data_dir = 'C:/Users/User/Desktop/ORVP/'
train = pd.read_csv(data_dir+'train.csv')
test = pd.read_csv(data_dir+'test.csv')

In [3]:
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap
def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap
# 對股價取log後再相減
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
def pressure_compute(df):
    sell_percent_with_mid_price1 = df["mid_price1"]/(df["mid_price1"]-df['ask_price1'])
    sell_percent_with_mid_price2 = df["mid_price1"]/(df["mid_price1"]-df['ask_price2'])
    buy_percent_with_mid_price1 = df["mid_price1"]/(df["mid_price1"]-df['bid_price1'])
    buy_percent_with_mid_price2 = df["mid_price1"]/(df["mid_price1"]-df['bid_price2'])
    df['sell_pressure'] = sell_percent_with_mid_price1/sell_percent_with_mid_price1.sum()*df['ask_size1']+ sell_percent_with_mid_price2/sell_percent_with_mid_price2.sum()*df['ask_size2']
    df['buy_pressure'] = buy_percent_with_mid_price1/buy_percent_with_mid_price1.sum()*df['bid_size1'] + buy_percent_with_mid_price2/buy_percent_with_mid_price2.sum()*df['bid_size2']
    df["pressure_ratio"] = np.log(df['sell_pressure']) - np.log(df['buy_pressure'])
def count_unique(series):
    return len(np.unique(series))
def read_train_test():
    
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)

    print(f'Our training set has {train.shape[0]} rows')
    return train, test


In [4]:
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    ##Price 
    ##(1) Weighted Price
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)

    df['log_return1'] = df.groupby('time_id')['wap1'].apply(log_return)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)

    ##(2) middle price 
    df["mid_price1"] = (df['ask_price1'] + df['bid_price1'])/2

    ##(3)price spread 
    df['price_spread1'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)

    ##(4) volume spread 
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])

    ##(5) imbalance 
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df["relative_wap_balance"] = (df['wap1'] - df['wap2'])/( df['wap1'] + df['wap2'])

    ##(6) Volume imbalance 
    df['depth_imbalance1'] = (df['ask_size1'] - df["bid_size1"])/(df['ask_size1'] + df["bid_size1"])
    df['depth_imbalance2'] = (df['ask_size2'] - df["bid_size2"])/(df['ask_size2'] + df["bid_size2"])

    ##(7) pressure_compute 
    pressure_compute(df)
    
    ##(8)volume imbalance 
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))



    create_feature_dict = {
        'wap1': [np.sum, np.max, np.min, np.median, np.std],
        'wap2': [np.sum, np.max, np.min, np.median, np.std],
        'wap3': [np.sum, np.max, np.min, np.median, np.std],
        'wap4': [np.sum, np.max, np.min, np.median, np.std],
        
        'log_return1': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'log_return2': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'log_return3': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'log_return4': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],

        "mid_price1":[np.sum, np.max, np.min, np.median, np.std],
        
        'price_spread1':[np.sum, np.max, np.min, np.median, np.std],
        'price_spread2':[np.sum, np.max, np.min, np.median, np.std],

        'bid_spread':[np.sum, np.max, np.min, np.median, np.std],
        'ask_spread':[np.sum, np.max, np.min, np.median, np.std],
        "bid_ask_spread":[np.sum, np.max, np.min, np.median, np.std],

        'wap_balance': [np.sum, np.max, np.min, np.median, np.std],
        "relative_wap_balance":[np.sum, np.max, np.min, np.median, np.std],

        'depth_imbalance1':[np.sum, np.max, np.min, np.median, np.std],
        'depth_imbalance2':[np.sum, np.max, np.min, np.median, np.std],

        'total_volume':[np.sum, np.max, np.min, np.median, np.std],
        'volume_imbalance':[np.sum, np.max, np.min, np.median, np.std],
        
    }
    create_feature_dict_time = {
        'log_return1': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'log_return2': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'log_return3': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'log_return4': [realized_volatility, np.sum, np.max, np.min, np.median, np.std],
    }

    
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(cf_dict, seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(cf_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict, seconds_in_bucket = 0, add_suffix = False)
    #df_feature_60 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 60, add_suffix = True)
    #df_feature_120 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 120, add_suffix = True)
    #df_feature_180 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 180, add_suffix = True)
    df_feature_240 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 240, add_suffix = True)
    #df_feature_300 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 300, add_suffix = True)
    #df_feature_360 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 360, add_suffix = True)
    #df_feature_420 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 420, add_suffix = True)
    df_feature_480 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 480, add_suffix = True)
    #df_feature_540 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 540, add_suffix = True)

    
    # Merge all
    #df_feature = df_feature.merge(df_feature_60, how = 'left', left_on = 'time_id_', right_on = 'time_id__60')
    #df_feature = df_feature.merge(df_feature_120, how = 'left', left_on = 'time_id_', right_on = 'time_id__120')
    #df_feature = df_feature.merge(df_feature_180, how = 'left', left_on = 'time_id_', right_on = 'time_id__180')
    df_feature = df_feature.merge(df_feature_240, how = 'left', left_on = 'time_id_', right_on = 'time_id__240')
    #df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    #df_feature = df_feature.merge(df_feature_360, how = 'left', left_on = 'time_id_', right_on = 'time_id__360')
    #df_feature = df_feature.merge(df_feature_420, how = 'left', left_on = 'time_id_', right_on = 'time_id__420')
    df_feature = df_feature.merge(df_feature_480, how = 'left', left_on = 'time_id_', right_on = 'time_id__480')
    #df_feature = df_feature.merge(df_feature_540, how = 'left', left_on = 'time_id_', right_on = 'time_id__540')

    # Drop unnecesary time_ids
    df_feature.drop([ 'time_id__240','time_id__480'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [5]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min, np.median, np.std],
        'order_count':[np.sum, np.max, np.min, np.median, np.std],
        'amount':[np.sum, np.max, np.min, np.median, np.std],
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility, np.sum, np.max, np.min, np.median, np.std],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min, np.median, np.std],
        'order_count':[np.sum, np.max, np.min, np.median, np.std],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(cf_dict, seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(cf_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict, seconds_in_bucket = 0, add_suffix = False)
    #df_feature_60 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 60, add_suffix = True)
    #df_feature_120 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 120, add_suffix = True)
    #df_feature_180 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 180, add_suffix = True)
    df_feature_240 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 240, add_suffix = True)
    #df_feature_300 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 300, add_suffix = True)
    #df_feature_360 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 360, add_suffix = True)
    #df_feature_420 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 420, add_suffix = True)
    df_feature_480 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 480, add_suffix = True)
    #df_feature_540 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 540, add_suffix = True)

    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)

        # PoV(Percent of Volume)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # vol vars    
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    #df_feature = df_feature.merge(df_feature_60, how = 'left', left_on = 'time_id_', right_on = 'time_id__60')
    #df_feature = df_feature.merge(df_feature_120, how = 'left', left_on = 'time_id_', right_on = 'time_id__120')
    #df_feature = df_feature.merge(df_feature_180, how = 'left', left_on = 'time_id_', right_on = 'time_id__180')
    df_feature = df_feature.merge(df_feature_240, how = 'left', left_on = 'time_id_', right_on = 'time_id__240')
    #df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    #df_feature = df_feature.merge(df_feature_360, how = 'left', left_on = 'time_id_', right_on = 'time_id__360')
    #df_feature = df_feature.merge(df_feature_420, how = 'left', left_on = 'time_id_', right_on = 'time_id__420')
    df_feature = df_feature.merge(df_feature_480, how = 'left', left_on = 'time_id_', right_on = 'time_id__480')
    #df_feature = df_feature.merge(df_feature_540, how = 'left', left_on = 'time_id_', right_on = 'time_id__540')

    # Drop unnecesary time_ids
    df_feature.drop(['time_id__240','time_id__480'], axis = 1, inplace = True)

    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

In [6]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 
            #'log_return1_realized_volatility_60', 'log_return2_realized_volatility_60', 
            #'log_return1_realized_volatility_120', 'log_return2_realized_volatility_120', 
            #'log_return1_realized_volatility_180', 'log_return2_realized_volatility_180', 
            'log_return1_realized_volatility_240', 'log_return2_realized_volatility_240', 
            #'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 
            #'log_return1_realized_volatility_360', 'log_return2_realized_volatility_360',
            #'log_return1_realized_volatility_420', 'log_return2_realized_volatility_420', 
            'log_return1_realized_volatility_480', 'log_return2_realized_volatility_480', 
            #'log_return1_realized_volatility_540', 'log_return2_realized_volatility_540',
            'trade_log_return_realized_volatility', #'trade_log_return_realized_volatility_60', 
            #'trade_log_return_realized_volatility_120', #'trade_log_return_realized_volatility_180',
            'trade_log_return_realized_volatility_240', #'trade_log_return_realized_volatility_300', 
            #'trade_log_return_realized_volatility_360', #'trade_log_return_realized_volatility_420',
            'trade_log_return_realized_volatility_480', #'trade_log_return_realized_volatility_540'
            ]
    

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'median' ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'median' ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

In [7]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

In [8]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   53.9s
C:\commfile\Anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
corr_matrix = train.corr()
train_corr = corr_matrix['target'].sort_values(ascending = False)
train_corr.to_csv(data_dir+'train_corr.csv',index=True)
train_list = train_corr[abs(train_corr.values) > 0.3].index
n_train = pd.concat([train.iloc[:,:2],train.loc[:,train_list]],axis=1)
n_train.to_csv(data_dir+'import.csv',index=False) 

In [12]:


# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   53.4s
C:\commfile\Anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  3.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [13]:
# replace by order sum (tau)
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
# train['size_tau_120'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_120'] )
# test['size_tau_120'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_120'] )
train['size_tau_240'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_240'] )
test['size_tau_240'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_240'] )
# train['size_tau_360'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_360'] )
# test['size_tau_360'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_360'] )
train['size_tau_480'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_480'] )
test['size_tau_480'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_480'] )

In [14]:
train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
train['size_tau2_480'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
test['size_tau2_480'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
# train['size_tau2_360'] = np.sqrt( 0.44/ train['trade_order_count_sum'] )
# test['size_tau2_360'] = np.sqrt( 0.44/ test['trade_order_count_sum'] )
train['size_tau2_240'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
test['size_tau2_240'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
# train['size_tau2_120'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
# test['size_tau2_120'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

# delta tau
train['size_tau2_d'] = train['size_tau2_480'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_480'] - test['size_tau2']

In [15]:
from sklearn.cluster import KMeans
# making agg features

train_p = train.copy()
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=2021).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(n_clusters):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)
mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

[1 4 5 6 1 1 6 5 3 6 4 4 5 5 1 1 1 6 5 5 7 4 1 1 7 1 1 5 7 5 7 5 5 1 7 7 5
 7 5 1 5 1 5 5 1 0 5 5 5 0 4 7 7 7 6 0 6 5 1 5 5 1 5 1 4 7 0 4 7 0 3 2 7 0
 0 1 0 4 7 0 0 5 1 1 0 0 7 7 1 0 1 5 5 5 5 1 1 7 1 4 1 5 1 4 1 5 1 4 5 0 5
 4]
[50, 55, 62, 75, 78, 83, 84, 86, 89, 90, 96, 97, 101, 124]
[0, 4, 5, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 66, 69, 72, 85, 94, 95, 100, 102, 108, 109, 111, 113, 115, 118, 120]
[81]
[8, 80]
[1, 10, 11, 22, 56, 73, 76, 87, 112, 116, 122, 126]
[2, 7, 13, 14, 19, 20, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 53, 64, 67, 68, 70, 93, 103, 104, 105, 107, 114, 119, 123, 125]
[3, 6, 9, 18, 61, 63]
[21, 27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 77, 82, 88, 98, 99, 110]


In [17]:
train = pd.merge(train,mat1,how='left',on='time_id')
test = pd.merge(test,mat2,how='left',on='time_id')

In [ ]:
import gc
del mat1,mat2
gc.collect()

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb

fold_n = 8

seed0=2021
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':500,
    'learning_rate': 0.2,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
# Function to early stop with root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False
# learning rate schedule
def learning_rate_decay(current_iter):
  base_learning_rate = params0.get('learning_rate') 
  
  lr = base_learning_rate * np.power(.998, current_iter) + base_learning_rate ** (current_iter/1000)
  if current_iter % 50 == 0:
    print(f'---------- learning_rate : {lr} ----------') 
  
  return lr 
def train_and_evaluate_lgb(train, test, params):
    # Hyperparammeters (just basic)
    
    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = fold_n, random_state = 2021, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=1501,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=50,
                          feval = feval_rmspe,
                          callbacks=[lgb.reset_parameter(learning_rate=learning_rate_decay)])
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val[features])
        # Predict the test set
        test_predictions += model.predict(test[features]) / fold_n

        lgb.plot_importance(model,max_num_features=20)
        

    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    lgb.plot_importance(model,max_num_features=20)

    # Return test predictions
    return test_predictions
# Traing and evaluate
predictions_lgb= train_and_evaluate_lgb(n_train, test,params0)
test['target'] = predictions_lgb
test[['row_id', 'target']].to_csv(data_dir+'submission.csv',index = False)

In [ ]:
# import matplotlib.pyplot as plt

# def train_and_evaluate(train, test):
#     # Hyperparammeters (optimized)
#     seed = 42
#     params = {
#         'learning_rate': 0.22,        
#         'lambda_l1': 2,
#         'lambda_l2': 7,
#         'num_leaves': 800,
#         'min_sum_hessian_in_leaf': 20,
#         'feature_fraction': 0.8,
#         'feature_fraction_bynode': 0.8,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 42,
#         'min_data_in_leaf': 700,
#         'max_depth': 4,
#         'seed': seed,
#         'feature_fraction_seed': seed,
#         'bagging_seed': seed,
#         'drop_seed': seed,
#         'data_random_seed': seed,
#         'objective': 'rmse',
#         'boosting': 'gbdt',
#         'verbosity': -1,
#         'n_jobs': -1,
#     }   
#     # learning rate schedule
#     def learning_rate_decay(current_iter):
#       base_learning_rate = params.get('learning_rate')
      
#       lr = base_learning_rate * np.power(.995, current_iter) + 0.08
#       if current_iter % 50 == 0:
#         print(f'---------- learning_rate : {lr} ----------') 
      
#       return lr 

#     # Split features and target
#     x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
#     y = train['target']
#     x_test = test.drop(['row_id', 'time_id'], axis = 1)
#     # Transform stock id to a numeric value
#     x['stock_id'] = x['stock_id'].astype(int)
#     x_test['stock_id'] = x_test['stock_id'].astype(int)
    
#     # Create out of folds array
#     oof_predictions = np.zeros(x.shape[0])
#     # Create test array to store predictions
#     test_predictions = np.zeros(x_test.shape[0])
#     # Create a KFold object
#     kfold = KFold(n_splits = 4, random_state = seed, shuffle = True)
#     # Iterate through each fold
#     for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
#         print(f'Training fold {fold + 1}')
#         x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
#         y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
#         # Root mean squared percentage error weights
#         train_weights = 1 / np.square(y_train)
#         val_weights = 1 / np.square(y_val)
#         train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
#         val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
#         model = lgb.train(params = params, 
#                           train_set = train_dataset, 
#                           valid_sets = [train_dataset, val_dataset], 
#                           num_boost_round = 3000, 
#                           early_stopping_rounds = 50, 
#                           verbose_eval = 100,
#                           feval = feval_rmspe,
#                           callbacks=[lgb.reset_parameter(learning_rate=learning_rate_decay)])

#         plt.figure(figsize=(12,6))
#         lgb.plot_importance(model, max_num_features=20)
#         plt.title("Feature importance")
#         plt.show()
#         # Add predictions to the out of folds array
#         oof_predictions[val_ind] = model.predict(x_val)
#         # Predict the test set
#         test_predictions += model.predict(x_test) / 4
        
#     rmspe_score = rmspe(y, oof_predictions)
#     print(f'Our out of folds RMSPE is {rmspe_score}')
#     # Return test predictions
#     return test_predictions

In [ ]:
# # Traing and evaluate
# test_predictions = train_and_evaluate(train, test)

 kaggle (baseline.ver1 learning rate 0.212) = Our out of folds RMSPE is 0.19834292014995092  
 kaggle (baseline_lgb_ver2 learning rate 0.211) = Our out of folds RMSPE is 0.19838470092173055  

In [ ]:
# # Save test predictions
# test['target'] = test_predictions
# test[['row_id', 'target']].to_csv('submission.csv',index = False)

# FFNN

In [ ]:
from numpy.random import seed
from sklearn.preprocessing import MinMaxScaler
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
import numpy as np
from keras import backend as K
import numpy.matlib

In [ ]:
def root_mean_squared_per_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))

In [ ]:
out_train = pd.read_csv(data_dir+'train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

#out_train[out_train.isna().any(axis=1)]
out_train = out_train.fillna(out_train.mean())
out_train.head()

# code to add the just the read data after first execution

# data separation based on knn ++
nfolds = 8 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]


lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

lineNumber = np.sort(lineNumber)[::-1]

for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
         # saves the values of index           

        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
colNames = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]
len(colNames)

In [ ]:
colNames

In [ ]:
from sklearn.preprocessing import QuantileTransformer
#colNames.remove('row_id')
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
for col in colNames:
    #print(col)
    qt = QuantileTransformer(random_state=2021,n_quantiles=2000, output_distribution='normal')
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)

In [ ]:
qt_train

In [ ]:
train_nn[['stock_id','time_id','target']]=train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']]=test[['stock_id','time_id']]

In [ ]:
# making agg features
from sklearn.cluster import KMeans
train_p = pd.read_csv(data_dir+'train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=n_clusters, random_state=2021).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(n_clusters):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train_nn.loc[train_nn['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test_nn.loc[test_nn['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

In [ ]:
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
import gc
train_nn = pd.merge(train_nn,mat1[nnn],how='left',on='time_id')
test_nn = pd.merge(test_nn,mat2[nnn],how='left',on='time_id')
# train_nn = pd.merge(train_nn,mat1,how='left',on='time_id')
# test_nn = pd.merge(test_nn,mat2,how='left',on='time_id')
# del mat1,mat2
# del train,test
# gc.collect()

In [ ]:
train_nn

## Base Model

In [ ]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})







def base_model():
    
    hidden_units = (128,64,32)
    stock_embedding_size = 24
    cat_data = train_nn['stock_id']

    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(884,), name='num_data')


    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        

    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [ ]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

## Prediction

In [ ]:
from sklearn import preprocessing, model_selection
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 8
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2021)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)
features_to_consider.remove('time_id')
features_to_consider.remove('target')
# features_to_consider.remove('row_id')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])

for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
    # NN   
    model = base_model()
    es = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=50, verbose=0,
            mode='min',restore_best_weights=True)

    plateau = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.2, patience=7, verbose=1,
            mode='min')# kfold based on the knn++ algorithm
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
        loss=root_mean_squared_per_error
    )
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['stock_id']    
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=1280,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
              verbose = 1)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    
    tt =scaler.transform(test_nn[features_to_consider].values)
    #test_nn[target_name] += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
    test_predictions_nn += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)
       
    counter += 1
    features_to_consider.append('stock_id')

In [ ]:
test_nn["row_id"] = test_nn["stock_id"].astype(str) + "-" + test_nn["time_id"].astype(str) 
test_nn[target_name] = (test_predictions_nn+predictions_lgb)/2

score = round(rmspe(y_true = train_nn[target_name].values, y_pred = train_nn[pred_name].values),5)
print('RMSPE {}: {} - Folds: {}'.format(model_name, score, scores_folds[model_name]))

display(test_nn[['row_id', target_name]].head(3))
test_nn[['row_id', target_name]].to_csv('submission2.csv',index = False)